## Análise de Dados usando o PySpark

### Neste notebook, foi feita uma analise de uma base de dados do Kaggle, sobre filmes e programas da Netflix.

Nesta analise contem:

- Valores unicos dos tipos de filmes e programas.
- Filmes que lançaram em 2020 no Brasil.
- Filmes que duram mais de 1 hora no Brasil.
- O total de filmes na Netflix.
- O total de programas de TV na Netflix.
- Total de filmes e progromas de TV que foram adicionados depois de 01/01/1997, foi quando a Netflix foi criada.
- Total de temporadas por série.
- A melhor série.

In [1]:
# Importando o Findspark que permite importar o Pyspark como uma biblioteca do Python
import findspark

In [2]:
# Importando o PySpark
import pyspark

In [3]:
# Importando todas as funções PySpark SQL
from pyspark.sql.functions import *

In [4]:
# Importando o SparkSession para trabalharmos com DataFrame
from pyspark.sql import SparkSession
spark = SparkSession.builder.getOrCreate()

#### Importando a base de dados

In [5]:
df = spark.read.csv("netflix_titles.csv", header=True, inferSchema=True)

#### Colunas do Dataframe

In [6]:
df.columns

['show_id',
 'type',
 'title',
 'director',
 'cast',
 'country',
 'date_added',
 'release_year',
 'rating',
 'duration',
 'listed_in',
 'description']

### Limpeza e preparação dos dados

In [7]:
# Apagando colunas que não iremos utilizar 
df1 = df.drop("show_id", "cast", "rating", "listed_in", "description")
df1.show(15)

+-------+-----+--------------------+--------------------+-----------------+------------+---------+
|   type|title|            director|             country|       date_added|release_year| duration|
+-------+-----+--------------------+--------------------+-----------------+------------+---------+
|TV Show|   3%|                null|              Brazil|  August 14, 2020|        2020|4 Seasons|
|  Movie| 7:19|   Jorge Michel Grau|              Mexico|December 23, 2016|        2016|   93 min|
|  Movie|23:59|        Gilbert Chan|           Singapore|December 20, 2018|        2011|   78 min|
|  Movie|    9|         Shane Acker|       United States|November 16, 2017|        2009|   80 min|
|  Movie|   21|      Robert Luketic|       United States|  January 1, 2020|        2008|  123 min|
|TV Show|   46|         Serdar Akar|              Turkey|     July 1, 2017|        2016| 1 Season|
|  Movie|  122|     Yasir Al Yasiri|               Egypt|     June 1, 2020|        2019|   95 min|
|  Movie| 

In [8]:
# Renomeando as colunas
df2 = df1.select(col("type").alias("tipo"), col("title").alias("titulo"), col("director").alias("diretor"), 
               col("country").alias("pais"), col("date_added").alias("data_add"),
               col("release_year").alias("ano_lancamento"), 
               col("duration").alias("duracao"))
df2.show(15)

+-------+------+--------------------+--------------------+-----------------+--------------+---------+
|   tipo|titulo|             diretor|                pais|         data_add|ano_lancamento|  duracao|
+-------+------+--------------------+--------------------+-----------------+--------------+---------+
|TV Show|    3%|                null|              Brazil|  August 14, 2020|          2020|4 Seasons|
|  Movie|  7:19|   Jorge Michel Grau|              Mexico|December 23, 2016|          2016|   93 min|
|  Movie| 23:59|        Gilbert Chan|           Singapore|December 20, 2018|          2011|   78 min|
|  Movie|     9|         Shane Acker|       United States|November 16, 2017|          2009|   80 min|
|  Movie|    21|      Robert Luketic|       United States|  January 1, 2020|          2008|  123 min|
|TV Show|    46|         Serdar Akar|              Turkey|     July 1, 2017|          2016| 1 Season|
|  Movie|   122|     Yasir Al Yasiri|               Egypt|     June 1, 2020|      

In [9]:
# Apaga os campos de todas as colunas que possuem valores nulos
df3 = df2.select("tipo", "titulo", "diretor", "pais", "data_add", "ano_lancamento", "duracao").na.drop("any")

df3.show(15)

+-------+------+--------------------+--------------------+-----------------+--------------+--------+
|   tipo|titulo|             diretor|                pais|         data_add|ano_lancamento| duracao|
+-------+------+--------------------+--------------------+-----------------+--------------+--------+
|  Movie|  7:19|   Jorge Michel Grau|              Mexico|December 23, 2016|          2016|  93 min|
|  Movie| 23:59|        Gilbert Chan|           Singapore|December 20, 2018|          2011|  78 min|
|  Movie|     9|         Shane Acker|       United States|November 16, 2017|          2009|  80 min|
|  Movie|    21|      Robert Luketic|       United States|  January 1, 2020|          2008| 123 min|
|TV Show|    46|         Serdar Akar|              Turkey|     July 1, 2017|          2016|1 Season|
|  Movie|   122|     Yasir Al Yasiri|               Egypt|     June 1, 2020|          2019|  95 min|
|  Movie|   187|      Kevin Reynolds|       United States| November 1, 2019|          1997|

### Exploração dos Dados

In [10]:
# Tipo primitivo dos dados
df3.select("titulo", "titulo", "pais", "ano_lancamento")

DataFrame[titulo: string, titulo: string, pais: string, ano_lancamento: string]

In [11]:
# Valores unicos da coluna Tipo
df3.select("tipo").distinct().show()

+-------+
|   tipo|
+-------+
|TV Show|
|  Movie|
+-------+



In [12]:
# Filmes que lançaram em 2020 no Brasil
df3.select("titulo", "pais", "ano_lancamento").where("pais == 'Brazil' and ano_lancamento == '2020'").show()

+--------------------+------+--------------+
|              titulo|  pais|ano_lancamento|
+--------------------+------+--------------+
|     A Queen Is Born|Brazil|          2020|
|Afonso Padilha: C...|Brazil|          2020|
|Anitta: Made In H...|Brazil|          2020|
|Dating Around: Br...|Brazil|          2020|
|          Double Dad|Brazil|          2020|
|Just Another Chri...|Brazil|          2020|
|Luccas Neto em: A...|Brazil|          2020|
|Maurício Meirelle...|Brazil|          2020|
|        Rich in Love|Brazil|          2020|
|The Beginning of ...|Brazil|          2020|
|Thiago Ventura: P...|Brazil|          2020|
+--------------------+------+--------------+



In [13]:
# Filmes que duram mais que 1 hora
df3.select("titulo", "pais", "duracao").where("duracao > '60' and pais == 'Brazil'").show()

+--------------------+------+-------+
|              titulo|  pais|duracao|
+--------------------+------+-------+
|Afonso Padilha: C...|Brazil| 63 min|
|        Alice Junior|Brazil| 87 min|
|All The Reasons T...|Brazil| 90 min|
|      Back and Forth|Brazil| 95 min|
|   Boy and the World|Brazil| 80 min|
|City of God: 10 Y...|Brazil| 69 min|
|  Couples Counseling|Brazil| 88 min|
|       DJ Cinderella|Brazil| 95 min|
|Edmilson Filho: N...|Brazil| 80 min|
|Felipe Neto: My L...|Brazil| 91 min|
|Fishtronaut: The ...|Brazil| 78 min|
|       Forever Chape|Brazil| 74 min|
|               Jonas|Brazil| 97 min|
|            Love.com|Brazil| 92 min|
|Luccas Neto em: A...|Brazil| 81 min|
|Luccas Neto in: S...|Brazil| 87 min|
|Luccas Neto in: T...|Brazil| 71 min|
|Marco Luque: Tamo...|Brazil| 73 min|
|Maurício Meirelle...|Brazil| 63 min|
|   Nothing to Lose 2|Brazil| 97 min|
+--------------------+------+-------+
only showing top 20 rows



In [14]:
# O total de filmes na Netflix
df3.select("tipo").where("tipo == 'Movie'").count()

5023

In [15]:
# O total de programas de TV na Netflix
df3.select("tipo").where("tipo == 'TV Show'").count()

169

In [16]:
# Total de filmes e progromas de TV que foram adicionados depois de 01/01/1997, foi quando a Netflix foi criada
df3.select("titulo", "pais", "data_add").where("data_add > 'January 1, 1997'").count()

3527

In [17]:
# Total de temporadas por série
print("Quantas séries da Netflix tem ou teve 1 temporada:", end=" ")
print(df3.select("duracao").where(f"duracao == '1 Season'").count())
for temp in range(2, 11):
    print(f"Quantas séries da Netflix tem ou teve {temp} temporadas:", end=" ")
    print(df3.select("duracao").where(f"duracao == '{temp} Seasons'").count())
    

Quantas séries da Netflix tem ou teve 1 temporada: 127
Quantas séries da Netflix tem ou teve 2 temporadas: 16
Quantas séries da Netflix tem ou teve 3 temporadas: 5
Quantas séries da Netflix tem ou teve 4 temporadas: 4
Quantas séries da Netflix tem ou teve 5 temporadas: 7
Quantas séries da Netflix tem ou teve 6 temporadas: 3
Quantas séries da Netflix tem ou teve 7 temporadas: 0
Quantas séries da Netflix tem ou teve 8 temporadas: 4
Quantas séries da Netflix tem ou teve 9 temporadas: 2
Quantas séries da Netflix tem ou teve 10 temporadas: 0


In [18]:
# Além de ser a unica série que teve 15 temporadas na Netflix, é a melhor série que existe!
# Agora o Supernatural tem na Prime Video.
df3.select("titulo", "duracao").where("duracao == '15 Seasons'").show()

+------------+----------+
|      titulo|   duracao|
+------------+----------+
|Supernatural|15 Seasons|
+------------+----------+

